Let's see how to compose two Results values.
[Results](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/results)

The Result type is typically used in monadic error-handling, which is often referred to as [Railway-oriented Programming](https://swlaschin.gitbooks.io/fsharpforfunandprofit/content/posts/recipe-part2.html) within the F# community.

In [8]:
// Define a simple type which has fields that can be validated
type Request =
    { Name: string
      Email: string }

// Define some logic for what defines a valid name.
//
// Generates a Result which is an Ok if the name validates;
// otherwise, it generates a Result which is an Error.
let validateName req =
    match req.Name with
    | null -> Error "No name found."
    | "" -> Error "Name is empty."
    | "bananas" -> Error "Bananas is not a name."
    | _ -> Ok req

// Similarly, define some email validation logic.
let validateEmail req =
    match req.Email with
    | null -> Error "No email found."
    | "" -> Error "Email is empty."
    | s when s.EndsWith("bananas.com") -> Error "No email from bananas.com is allowed."
    | _ -> Ok req

let validateRequest reqResult =
    reqResult
    |> Result.bind validateName
    |> Result.bind validateEmail

let test() =
    // Now, create a Request and pattern match on the result.
    let req1 = { Name = "Phillip"; Email = "phillip@contoso.biz" }
    let res1 = validateRequest (Ok req1)
    match res1 with
    | Ok req -> printfn $"My request was valid! Name: {req.Name}, Email {req.Email}"  
    | Error e -> printfn $"Error: {e}"

    let req2 = { Name = "Phillip"; Email = "phillip@bananas.com" }
    let res2 = validateRequest (Ok req2)
    match res2 with
    | Ok req -> printfn $"My request was valid! Name: {req.Name}, Email {req.Email}"  
    | Error e -> printfn $"Error: {e}"

    let req3 = { Name = ""; Email = "phillip@bananas.com" }
    let res3 = validateRequest (Ok req3)
    match res3 with
    | Ok req -> printfn $"My request was valid! Name: {req.Name}, Email {req.Email}"  
    | Error e -> printfn $"Error: {e}"

In [9]:
test()

My request was valid! Name: Phillip, Email phillip@contoso.biz
Error: No email from bananas.com is allowed.
Error: Name is empty.


Notice that for the 3rd request, even though there were 2 errors, only the first one was reported. This is an example of a short circuiting behavior. 